# Tratamiento de datos nulos

La diferencia entre los datos que se encuentran en muchos tutoriales y los datos del mundo real es que los datos del mundo real rara vez están limpios y homogéneos.
En particular, **muchos conjuntos de datos interesantes tendrán alguna cantidad de datos faltantes**.
Para complicar aún más las cosas, diferentes fuentes de datos pueden indicar los datos que faltan de diferentes maneras.

En esta sección, discutiremos algunas consideraciones generales para los datos *missing* o faltantes, discutiremos cómo Pandas elige representarlos, y demostraremos algunas herramientas incorporadas de Pandas para manejar los datos faltantes (missings) en Python.
**Nos referiremos a los datos faltantes en general como valores *nulos*, *missing*, *NaN* o *NA* .



## "Buenas prácticas" para los datos faltantes


Hay una serie de esquemas que se han desarrollado para indicar la presencia de datos que faltan en una tabla o DataFrame.
Generalmente, giran en torno a una de dos estrategias: **utilizar una *máscara* que indique globalmente los valores que faltan, o elegir un *valor centinela* que indique una entrada que falta**.

En el enfoque de enmascaramiento, **la máscara** puede ser una matriz booleana completamente separada, o puede implicar la apropiación de un bit en la representación de datos para indicar localmente el estado nulo de un valor.

En el enfoque centinela, el valor centinela **puede ser alguna estrategia específica de los datos, como indicar un valor entero faltante con -9999** o algún patrón de bits específico, o puede ser una convención más global, como indicar un valor de punto flotante faltante con **NaN (Not a Number)**.

Ninguno de estos enfoques está exento de inconvenientes: el uso de una matriz de máscaras separada requiere la asignación de una matriz booleana adicional, lo que añade una sobrecarga tanto de almacenamiento como de cálculo. Un valor centinela reduce el rango de valores válidos que pueden representarse y puede requerir una lógica adicional (a menudo no optimizada) en la aritmética de la CPU y la GPU. Los valores especiales comunes como NaN no están disponibles para todos los tipos de datos.

Como en la mayoría de los casos en los que no existe una opción universalmente óptima, diferentes lenguajes y sistemas utilizan diferentes convenciones.
**Por ejemplo, el lenguaje R utiliza patrones de bits reservados dentro de cada tipo de datos como valores centinela que indican la ausencia de datos**, mientras que el sistema SciDB utiliza un byte adicional adjunto a cada celda que indica un estado NA.

## Missing Data en Pandas

La forma en que Pandas **maneja los valores faltantes `estaba` limitada por su dependencia del paquete NumPy**, que no tiene una noción incorporada de valores NA para los tipos de datos que no son de punto flotante. Veremos mejoras con la llegada de la versión 2.0.

Pandas podría haber seguido el ejemplo de **R especificando patrones de bits para cada tipo de datos individual** para indicar la nulidad, pero este enfoque resulta ser bastante difícil de manejar.
Mientras que R contiene cuatro tipos de datos básicos, **NumPy soporta *mucho* más que esto**: por ejemplo, mientras que R tiene un único tipo de entero, NumPy soporta *catorce* tipos de enteros básicos una vez que se tienen en cuenta las precisiones disponibles, la significación y la codificación.
**Reservar un patrón de bits específico en todos los tipos disponibles de NumPy llevaría a una cantidad inmanejable de sobrecarga** en las operaciones especiales para varios tipos, probablemente incluso requiriendo un nuevo fork del paquete NumPy. Además, para los tipos de datos más pequeños (como los enteros de 8 bits), sacrificar un bit para usarlo como máscara reducirá significativamente el rango de valores que puede representar.

**NumPy tiene soporte para arrays enmascarados** - es decir, arrays que tienen un array de máscara booleana separado para marcar los datos como "buenos" o "malos".
Pandas podría haber derivado de esto, pero la sobrecarga en el almacenamiento, el cálculo y el mantenimiento del código hace que sea una opción poco atractiva.

Con estas limitaciones en mente, **Pandas eligió usar centinelas para los datos perdidos**, y además eligió usar dos valores nulos ya existentes en Python: el valor especial de punto flotante **``NaN``, y el objeto Python ``None``**.
Esta elección tiene algunos efectos secundarios, como veremos, pero en la práctica acaba siendo un buen compromiso en la mayoría de los casos de interés.

### ``None``: Pythonic missing data

El primer valor centinela utilizado por Pandas es **``None``**, un objeto singleton de Python que se utiliza a menudo para los datos que faltan en el código de Python.
Debido a que es un objeto de Python, ``None`` no se puede utilizar en cualquier array arbitrario de NumPy/Pandas, sino sólo en arrays con tipo de datos ``'object'`` (es decir, arrays de objetos de Python):

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

Este ``dtype=object`` significa que la mejor representación de tipo común que NumPy puede inferir para los contenidos del array es que son objetos de Python.
Aunque este tipo de matriz de objetos es útil para algunos propósitos, cualquier operación sobre los datos se hará a nivel de Python, con mucha más sobrecarga que las operaciones típicamente rápidas vistas para las matrices con tipos nativos:

In [ ]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

El uso de objetos Python en un array también significa que si realizas agregaciones como ``sum()`` o ``min()`` en un array con un valor ``None``, generalmente obtendrás un error:

In [ ]:
vals1.sum()

Esto refleja el hecho de que la suma entre un entero y ``None`` no está definida.

### ``NaN``: Missing datos numéricos

La otra representación de datos faltantes, ``NaN`` (acrónimo de *Not a Number*), es diferente; es un valor especial de punto flotante reconocido por todos los sistemas que utilizan la representación estándar de punto flotante IEEE:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

Fíjate en que NumPy eligió un tipo nativo de punto flotante para este array: esto significa que **a diferencia del array de objetos de antes, este array soporta operaciones rápidas empujadas en el código compilado.**
Debes tener en cuenta que ``NaN`` es un poco como un virus de datos - infecta cualquier otro objeto que toque.
Independientemente de la operación, el resultado de la aritmética con ``NaN`` será otro ``NaN``:

In [ ]:
1 + np.nan

In [ ]:
0 *  np.nan

In [ ]:
vals2

Ten en cuenta que esto significa que los agregados sobre los valores están bien definidos (es decir, no dan lugar a un error) pero no siempre son útiles:

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

NumPy proporciona algunas agregaciones especiales que ignoran estos valores perdidos:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

Ten en cuenta que ``NaN`` es específicamente un valor de punto flotante; no hay un valor NaN equivalente para enteros, cadenas u otros tipos.

### NaN y None en Pandas

Tanto ``NaN`` como ``None`` tienen su lugar, y Pandas está construido para manejar los dos casi indistintamente, convirtiendo entre ellos cuando sea apropiado:

In [ ]:
pd.Series([1, np.nan, 2, None])

Para los tipos que no tienen un valor centinela disponible, Pandas realiza automáticamente una conversión de tipo cuando hay valores NA.
Por ejemplo, **si establecemos un valor en un array de enteros a ``np.nan``, se convertirá automáticamente a un tipo de punto flotante para acomodar el NA**:

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

Observe que **además de convertir el array de enteros a punto flotante, Pandas convierte automáticamente el ``None`` a un valor ``NaN``**.
(Ten en cuenta que hay una propuesta para añadir un entero nativo NA a Pandas en el futuro; en el momento de escribir esto, no se ha incluido).

Mientras que este tipo de magia puede parecer un poco deficiente en comparación con el enfoque más unificado de los valores NA en lenguajes específicos como R, el enfoque de centinela/casting de Pandas funciona bastante bien en la práctica y en mi experiencia sólo causa problemas en raras ocasiones.

La siguiente tabla lista de términos de upcasting en Pandas cuando se introducen valores NA:

|Typeclass     | Conversión cuando tiene NAs | NA Valor Centinela      |
|--------------|-----------------------------|------------------------|
| ``floating`` | Sin cambio                   | ``np.nan``             |
| ``object``   | Sin cambio                   | ``None`` o ``np.nan`` |
| ``integer``  | Cambia a ``float64``         | ``np.nan``             |
| ``boolean``  | Cambia a ``object``          | ``None`` o ``np.nan`` |

__Ten en cuenta que en Pandas, los datos de cadena siempre se almacenan con un dtype ``object``.__

## Operar con valores nulos

Como hemos visto, **Pandas trata ``None`` y ``NaN`` como esencialmente intercambiables para indicar valores nulos o missing.**
Para facilitar este hecho, existen varios métodos útiles para detectar, eliminar y reemplazar valores nulos en las estructuras de datos de Pandas.
Estos son:

- ``isnull()``: Generar una máscara booleana que indique los valores que faltan
- ``notnull()``: El contrario de ``isnull()``
- ``dropna()``: Devuelve una versión filtrada de los datos
- ``fillna()``: Devuelve una copia de los datos con los valores missing rellenados o imputados

Veremos unos ejemplos de estas funciones.

### Detectar valores null

Las estructuras de datos de Pandas tienen dos métodos útiles para detectar datos nulos: ``isnull()`` y ``notnull()``.
Cualquiera de ellos devolverá una máscara booleana sobre los datos. Por ejemplo:

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data

In [ ]:
data.isnull()

In [ ]:
data.notnull()

 Las máscaras booleanas se pueden utilizar directamente como índice de ``Series`` o ``DataFrame``:

In [ ]:
data[data.notnull()]

In [ ]:
data[~data.isnull()]

Los métodos ``isnull()`` y ``notnull()`` producen resultados booleanos similares para los ``DataFrame``.

### Quitando valores null

Además del enmascaramiento utilizado anteriormente, existen los métodos de gran utilidad como, ``dropna()``
(que elimina los valores NA) y ``fillna()`` (que rellena los valores NA). Para una ``Serie``,
el resultado es sencillo:

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])

In [ ]:
data.dropna()
#data.dropna(inplace=True)

In [ ]:
data

Para un ``DataFrame``, hay más opciones.
Veamos el siguiente ``DataFrame``:

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

In [ ]:
df.info()

No podemos eliminar valores individuales de un ``DataFrame``; sólo podemos eliminar filas o columnas completas.
Dependiendo de la aplicación, se puede querer una cosa u otra, por lo que ``dropna()`` da una serie de opciones para un ``DataFrame``.

Por defecto, ``dropna()`` eliminará todas las __filas__ en las que esté presente *cualquier* valor nulo:

In [ ]:
df.dropna()

Alternativamente, se puede dejar los valores NA a lo largo de un eje diferente; ``axis=1`` deja todas las columnas que contienen un valor nulo:

In [ ]:
df.dropna(axis=1)
# df.dropna(axis='columns')

Pero esto deja caer algunos datos buenos también; se podría estar más interesado en dejar caer filas o columnas con *todos* los valores NA, o una mayoría de valores NA.
Esto se puede especificar a través de los parámetros ``how`` o ``thresh``, que permiten un control fino del número de nulos a permitir.

El valor por defecto es ``how='any'``, de forma que cualquier fila o columna (dependiendo de la palabra clave ``axis``) que contenga un valor nulo será descartada.
También se puede especificar ``how='all'``, que sólo eliminará las filas/columnas que sean *todos* valores nulos:

In [ ]:
df[3] = np.nan
df

In [ ]:
df.dropna(axis='columns', how='all')

Para un control más preciso, el parámetro ``thresh`` permite especificar un número mínimo de valores no nulos para la fila/columna que debe conservarse:

In [ ]:
df

In [ ]:
df.dropna(axis='rows', thresh=3)

In [ ]:
df.dropna(axis='rows', thresh=len(df.columns)*0.75)

In [ ]:
df.dropna(axis='columns', thresh=len(df)*0.75)

Aquí se han eliminado la primera y la última fila, porque sólo contienen dos valores no nulos.

### Rellenar valores nulos

A veces, en lugar de eliminar los valores NA, se prefiere sustituirlos por un valor válido.
Este valor podría ser un número único como el cero, o podría ser algún tipo de imputación o interpolación de los valores buenos.
Se podría hacer esto en el lugar usando el método ``isnull()`` como máscara, pero como es una operación tan común Pandas proporciona el método ``fillna()``, que devuelve una copia del array con los valores nulos reemplazados.

Considera la siguiente ``Serie``:

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

Podemos rellenar las entradas NA con un único valor, como por ejemplo el cero:

In [ ]:
data.fillna(0)

Podemos especificar un forward-fill para propagar el valor anterior hacia adelante:

In [ ]:
# forward-fill CUIDADO DEPRECATED!
data.fillna(method='ffill')

In [ ]:
# forward-fill 2.0
data.ffill()

O podemos especificar un relleno para propagar los siguientes valores hacia atrás:

In [ ]:
# back-fill CUIDADO DEPRECATED!
data.fillna(method='bfill')

In [ ]:
# back-fill 2.0
data.bfill()

Para los ``DataFrame``, las opciones son similares, pero también podemos especificar un ``eje`` a lo largo del cual tienen lugar los rellenos:

In [ ]:
df

In [ ]:
df[1]

In [ ]:
df.fillna(df[1].mean())

In [ ]:
df

In [ ]:
df.fillna(method='bfill', axis=1) #deprecated in 2.0

In [ ]:
df.bfill(axis=1)

Observa que si un valor anterior no está disponible durante un llenado hacia adelante, el valor NA permanece.

# Indexing Jerárquico

Hasta ahora nos hemos centrado principalmente en datos unidimensionales y bidimensionales, almacenados en objetos Pandas ``Series`` y ``DataFrame``, respectivamente.
A menudo es útil ir más allá y almacenar datos de mayor dimensión, es decir, datos indexados por más de una o dos claves.
Aunque Pandas proporciona objetos ``Panel`` y ``Panel4D`` que manejan de forma nativa datos tridimensionales y cuatridimensionales, un patrón mucho más común en la práctica es hacer uso de la *indexación jerárquica* (también conocida como *multiindexación*) para incorporar múltiples *niveles de índice* dentro de un único índice.
De este modo, los datos de mayor dimensión pueden representarse de forma compacta dentro de los conocidos objetos unidimensionales ``Series`` y bidimensionales ``DataFrame``.

En esta sección, exploraremos la creación directa de los objetos ``MultiIndex``, las consideraciones a tener en cuenta a la hora de indexar, trocear y calcular las estadísticas de los datos con índices múltiples, y las rutinas útiles para convertir entre las representaciones simples y jerárquicas de los datos.



In [ ]:
import pandas as pd
import numpy as np

## Una Serie multiindexada

Empecemos por considerar cómo podríamos representar datos bidimensionales dentro de una ``Serie`` unidimensional.
Para concretar, consideraremos una serie de datos donde cada punto tiene un carácter y una clave numérica.

### La forma mala

Supongamos que quieres rastrear datos sobre estados de dos años diferentes.
Usando las herramientas de Pandas que ya hemos cubierto, podrías estar tentado a usar simplemente tuplas de Python como claves:

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

Con este esquema de indexación, se puede indexar o hacer slicing directamente la serie en función de este índice múltiple:

In [ ]:
pop[('California', 2010):('Texas', 2000)]

Pero la comodidad termina ahí. Por ejemplo, si necesitas seleccionar todos los valores de 2010, tendrás que hacer un poco de compensión de listas desordenada (y potencialmente lenta) para lograrlo:

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]]

Esto produce el resultado deseado, pero no es tan limpio (o tan eficiente para los grandes conjuntos de datos) como la sintaxis de slicing que hemos llegado a amar en Pandas <3  

### La forma buena: Pandas MultiIndex

Afortunadamente, Pandas proporciona una forma mejor.
Nuestra indexación basada en tuplas es esencialmente un multiíndice rudimentario, y el tipo Pandas ``MultiIndex`` nos da el tipo de operaciones que deseamos tener.
Podemos crear un multiíndice a partir de las tuplas de la siguiente manera:

In [ ]:
index = pd.MultiIndex.from_tuples(index)
index

Observe que el ``MultiIndex`` contiene múltiples *niveles* de indexación -en este caso, los nombres de los estados y los años-, así como múltiples *etiquetas* para cada punto de datos que codifican estos niveles.

Si volvemos a indexar nuestra serie con este ``MultiIndex``, veremos la representación jerárquica de los datos:

In [ ]:
pop = pop.reindex(index)
pop

Aquí las dos primeras columnas de la representación ``Series`` muestran los valores de los índices múltiples, mientras que la tercera columna muestra los datos.
Observa que faltan algunas entradas en la primera columna: en esta representación de índices múltiples, cualquier entrada en blanco indica el mismo valor que la línea superior.

Ahora, para acceder a todos los datos para los que el segundo índice es 2010, podemos simplemente utilizar la notación de ``slicing`` de Pandas:

In [ ]:
pop[:, 2010]

El resultado es un array indexado individualmente con sólo las claves que nos interesan.
Esta sintaxis es mucho más sencilla (¡y la operación es mucho más eficiente!) que la solución de indexación múltiple basada en tuplas con la que empezamos.
A continuación, discutiremos este tipo de operación de indexación en datos indexados jerárquicamente.

### MultiIndex como dimensión extra

Puedes notar algo más aquí: podríamos fácilmente haber almacenado los mismos datos usando un simple ``DataFrame`` con etiquetas de índice y columna.
De hecho, Pandas está construido con esta equivalencia en mente. El método ``unstack()`` convertirá rápidamente una ``Serie`` con índices múltiples en un ``DataFrame`` con índices convencionales:

In [ ]:
pop_df = pop.unstack()
pop_df

Naturalmente, el método ``stack()`` proporciona la operación contraria:

In [ ]:
pop_df.stack()

Al ver esto, te preguntarás por qué nos molestamos en utilizar la indexación jerárquica.
La razón es sencilla: igual que hemos podido utilizar la multiindexación para representar datos bidimensionales dentro de una ``Serie`` unidimensional, también podemos utilizarla para representar datos de tres o más dimensiones en una ``Serie`` o ``DataFrame``.
Cada nivel extra en un multiíndice representa una dimensión extra de datos; aprovechar esta propiedad nos da mucha más flexibilidad en los tipos de datos que podemos representar. Concretamente, podríamos querer añadir otra columna de datos demográficos para cada estado en cada año (digamos, población menor de 18 años); con un ``MultiIndex`` esto es tan fácil como añadir otra columna al ``DataFrame``:

In [ ]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

Aquí calculamos la fracción de personas menores de 18 años por año, dados los datos anteriores:

In [ ]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

Esto nos permite manipular y explorar fácil y rápidamente incluso los datos de alta dimensión.

## Métodos de creación de multiindices

La forma más directa de construir una ``Serie`` o un ``DataFrame`` con índices múltiples es simplemente pasar una lista de dos o más matrices de índices al constructor. Por ejemplo:

In [ ]:
np.random.seed(6)
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

El trabajo de crear el ``MultiIndex`` se hace en segundo plano.

Del mismo modo, si se pasa un diccionario con tuplas apropiadas como claves, Pandas lo reconocerá automáticamente y utilizará un ``MultiIndex`` por defecto:

In [ ]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

Sin embargo, a veces es útil crear explícitamente un ``MultiIndex``; aquí veremos un par de estos métodos.

### Constructores explícitos de MultiIndex

Para una mayor flexibilidad en la construcción del índice, puedes utilizar los métodos constructores de la clase ``pd.MultiIndex``.
Por ejemplo, como hemos hecho antes, puedes construir el ``MultiIndex`` a partir de una simple lista de arrays con los valores del índice dentro de cada nivel:

In [ ]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

Se puede construir a partir de una lista de tuplas que dan los valores de los índices múltiples de cada punto:

In [ ]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

Incluso puedes construirlo a partir de un producto cartesiano de índices individuales:

In [ ]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

Del mismo modo, puedes construir el ``MultiIndex`` directamente utilizando su codificación interna pasando ``niveles`` (una lista de listas que contienen los valores de índice disponibles para cada nivel) y ``etiquetas`` (una lista de listas que hacen referencia a estas etiquetas):

In [ ]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 0, 1, 1]])

Cualquiera de estos objetos puede pasarse como argumento ``index`` cuando se crea una ``Serie`` o ``Dataframe``, o pasarse al método ``reindex`` de una ``Serie`` o ``DataFrame`` existente.

### Nombres de niveles de MultiIndex

A veces es conveniente nombrar los niveles del ``MultiIndex``.
Esto se puede conseguir pasando el argumento ``names`` a cualquiera de los constructores de ``MultiIndex``, o estableciendo el atributo ``names`` del índice a posteriori:

In [ ]:
pop.index.names = ['state', 'year']
pop

Con conjuntos de datos más complicados, esta puede ser una forma útil de seguir el significado de varios valores de índice.

### MultiIndex para columnas

En un ``DataFrame``, las filas y columnas son completamente simétricas en propiedades, y al igual que las filas pueden tener múltiples niveles de índices, las columnas también pueden tener múltiples niveles.
Considere lo siguiente, que es una maqueta de algunos datos médicos (algo realistas):.

In [ ]:
# Índices jerárquicos y columnas
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# Creamos algunos datos aleatorios
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# Creamos el dataframe
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

Aquí vemos que la multiindexación de filas y columnas puede ser *muy* útil.
Se trata de datos fundamentalmente cuatridimensionales, donde las dimensiones son el sujeto, el tipo de medición, el año y el número de visita.
Con esto en su lugar podemos, por ejemplo, indexar la columna de nivel superior por el nombre de la persona y obtener un ``DataFrame`` completo que contenga sólo la información de esa persona:

In [ ]:
health_data['Guido']

Para los registros complicados que contienen múltiples mediciones etiquetadas a lo largo de múltiples tiempos para muchos sujetos (personas, países, ciudades, etc.), el uso de filas y columnas jerárquicas puede ser extremadamente conveniente.

## Indexado y Slicing un MultiIndex

La indexación y el slicing en un ``MultiIndex`` está diseñado para ser intuitivo, y ayuda si piensas en los índices como dimensiones añadidas.
Primero veremos la indexación de ``Series`` con índices múltiples, y luego la de ``DataFrame`` con índices múltiples.

### Series con multiindice

Considera la ``Serie`` de poblaciones de estados multi-indexada que vimos anteriormente:

In [ ]:
pop

Podemos acceder a elementos individuales mediante la indexación con términos múltiples:

In [ ]:
pop['California', 2000]

El ``MultiIndex`` también admite la *indexación parcial*, es decir, la indexación de sólo uno de los niveles del índice.
El resultado es otra ``Serie``, con los índices de nivel inferior mantenidos:

In [ ]:
pop['California']

El corte parcial también está disponible, siempre que el ``MultiIndex`` esté ordenado

In [ ]:
pop.loc['California':'New York']

Con los índices ordenados, se puede realizar una indexación parcial en los niveles inferiores pasando un trozo vacío en el primer índice:

In [ ]:
pop[:, 2000]

También funcionan otros tipos de indexación y selección; por ejemplo, la selección basada en máscaras booleanas:

In [ ]:
pop[pop > 22000000]

También funciona la selección basada en la indexación sofisticada o ``fancy``:

In [ ]:
pop[['California', 'Texas']]

### DataFrames con multiindex

Un ``DataFrame`` de índice múltiple se comporta de manera similar.
Consideremos nuestro ``DataFrame`` médico de antes:

In [ ]:
health_data

Recuerda que las columnas son primarias en un ``DataFrame``, y que la sintaxis utilizada para las ``Series`` con índices múltiples se aplica a las columnas.
Por ejemplo, podemos recuperar los datos de la frecuencia cardíaca de Guido con una simple operación:

In [ ]:
health_data['Guido', 'HR']

Además, al igual que en el caso de un solo índice, podemos utilizar el ``loc``, ``iloc``.

In [ ]:
health_data.iloc[:3, :3]

Estos indexadores proporcionan una vista similar a un array de los datos bidimensionales subyacentes, pero a cada índice individual en ``loc`` o ``iloc`` se le puede pasar una tupla de múltiples índices. Por ejemplo:

In [ ]:
health_data.loc[:, ('Bob', 'HR')]

Trabajar con slices dentro de estas tuplas de índice no es especialmente conveniente; intentar crear un slice dentro de una tupla conducirá a un error de sintaxis:

In [ ]:
health_data.loc[(:, 1), (:, 'HR')]

Se podría evitar esto construyendo la porción o slice deseada/o explícitamente usando la función incorporada de Python ``slice()``, pero una mejor manera en este contexto es usar un objeto ``IndexSlice``, que Pandas proporciona precisamente para esta situación.
Por ejemplo:

In [ ]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

Hay muchas formas de interactuar con los datos en las ``Series`` y los ``DataFrame`` multi-indexados, y al igual que con muchas de las herramientas de este script, la mejor manera de familiarizarse con ellas es probarlas.

## Reorganización de los multiíndices

Una de las claves para trabajar con datos de índice múltiple es saber cómo transformar los datos de forma efectiva.
Hay una serie de operaciones que conservarán toda la información del conjunto de datos, pero la reorganizarán con el fin de realizar diversos cálculos.
Vimos un breve ejemplo de esto en los métodos ``stack()`` y ``unstack()``, pero hay muchas más formas de controlar finamente el reordenamiento de los datos entre índices jerárquicos y columnas, y las exploraremos aquí.

### Índices ordenados y no ordenados

Antes hemos mencionado brevemente una advertencia, pero deberíamos hacer más hincapié en ella aquí.
**Muchas de las operaciones de corte de ``MultiIndex`` fallarán si el índice no está ordenado.
Echemos un vistazo a esto.**

Empezaremos creando algunos datos simples de índice múltiple donde los índices no están *ordenados lexográficamente*:

In [ ]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

Si intentamos tomar una porción o slice parcial de este índice, se producirá un error:

In [ ]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

Aunque no está del todo claro en el mensaje de error, esto es el resultado de que el MultiIndex no esté ordenado.
Por varias razones, los cortes parciales y otras operaciones similares requieren que los niveles del ``MultiIndex`` estén ordenados (es decir, lexográficos).
Pandas proporciona un número de métodos de gran utilidad para realizar este tipo de ordenación; ejemplos son los métodos ``sort_index()`` y ``sortlevel()`` del ``DataFrame``.
Aquí utilizaremos el más sencillo, ``sort_index()``:

In [ ]:
data = data.sort_index()
data

Con el índice ordenado de esta manera, el corte parcial funcionará como se espera:

In [ ]:
data['a':'b']

### Stacking y unstacking índices

Como hemos visto brevemente antes, es posible convertir un conjunto de datos de un índice múltiple apilado a una representación bidimensional simple, especificando opcionalmente el nivel a utilizar:

In [ ]:
pop.unstack(level=0)

In [ ]:
pop.unstack(level=1)

Lo contrario de ``unstack()`` es ``stack()``, que aquí se puede utilizar para recuperar la serie original:

In [ ]:
pop.unstack().stack()

### Fijación y restablecimiento de índices

Otra forma de reorganizar los datos jerárquicos es convertir las etiquetas de los índices en columnas; esto se puede lograr con el método ``reset_index``.
Si se llama a este método en el diccionario de población, se obtendrá un ``DataFrame`` con una columna *estado* y *año* que contiene la información que antes estaba en el índice.
Para mayor claridad, podemos especificar opcionalmente el nombre de los datos para la representación de la columna:

In [ ]:
pop_flat = pop.reset_index(name='population')
pop_flat

A menudo, cuando se trabaja con datos en el mundo real, los datos de entrada en bruto tienen este aspecto y es útil construir un ``MultiIndex`` a partir de los valores de las columnas.
Esto se puede hacer con el método ``set_index`` del ``DataFrame``, que devuelve un ``DataFrame`` con índice múltiple:

In [ ]:
pop_flat.set_index(['state', 'year'])

En la práctica, encuentro que este tipo de reindexación es uno de los patrones más útiles cuando me encuentro con conjuntos de datos del mundo real.

## Agregaciones de datos en multiíndices

Ya hemos visto que Pandas tiene métodos de agregación de datos incorporados, como ``media()``, ``suma()`` y ``máx()``.
Para los datos indexados jerárquicamente, se les puede pasar un parámetro de ``nivel`` que controla el subconjunto de datos sobre el que se calcula el agregado.

Por ejemplo, volvamos a nuestros datos de salud:

In [ ]:
health_data

En las versiones 1.0 de Pandas podríamos haber hecho lo siguiente; Promediar las mediciones en las dos visitas de cada año. Podemos hacerlo nombrando el nivel de índice que nos gustaría explorar, en este caso el año:

In [ ]:
data_mean = health_data.mean(level='year')
data_mean

Pero esto ya solo es posible con el ``GroupBy``, que veremos próximamente, lo que sí podemos hacer es utilizar el método con el argumento axis para cambiar el enfoque con índices de columna o de fila:

In [ ]:
data_mean = health_data.mean(axis=1)
data_mean

In [ ]:
data_mean2 = health_data.mean(axis=0)
data_mean2

Haciendo uso además de la palabra clave ``axis``, podemos tomar también la media entre los niveles de las columnas:

In [ ]:
print(data_mean.mean(),data_mean2.mean())

## Panel Data

Pandas tiene otras estructuras de datos fundamentales que aún no hemos discutido, a saber, los objetos ``pd.Panel`` y ``pd.Panel4D``.
Estos pueden ser considerados, respectivamente, como generalizaciones tridimensionales y cuatridimensionales de las estructuras (unidimensionales) ``Series`` y (bidimensionales) ``DataFrame``.
Una vez que te hayas familiarizado con la indexación y manipulación de datos en una ``Serie`` y un ``DataFrame``, el ``Panel`` y el ``Panel4D`` son relativamente sencillos de utilizar.
En particular, los indexadores ``loc`` y ``iloc`` se extienden fácilmente a estas estructuras de mayor dimensión.

No cubriremos más estas estructuras de panel en este curso, ya que he encontrado en la mayoría de los casos que la multi-indexación es una representación más útil y conceptualmente más simple para los datos de mayor dimensión.
Además, los datos de panel son fundamentalmente una representación de datos densos, mientras que la multiindexación es fundamentalmente una representación de datos dispersos.
A medida que aumenta el número de dimensiones, la representación densa puede resultar muy ineficiente para la mayoría de los conjuntos de datos del mundo real.
Sin embargo, para las aplicaciones especializadas ocasionales, estas estructuras pueden ser útiles.
